In [13]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import numpy as np
from scipy.stats import poisson, norm
import random
init_notebook_mode(connected=True)


In [14]:
cpi = 0.02
old_rate = cpi + 0.02
new_rate = cpi
market_rate = 0.025
cont_rate = np.log(1+market_rate)
starting_rent = 2000
years = 10
turnover_rate = .2/12 # per month

def tenancy_end(m):
    end = False
    if m > 12 and random.random() < turnover_rate:
        end = True
    return end

In [15]:
market_rv = norm()
market_rv.pdf(0,loc=1.0)

TypeError: pdf() got an unexpected keyword argument 'loc'

In [3]:
market_rent = starting_rent
controlled_rent = starting_rent
market_rents = []
all_months = np.linspace(1.0,years*12,num=years*12)


for m in all_months:
    market_rent = starting_rent*np.exp(m*cont_rate/12)
    market_rents.append(market_rent)

def controlled_rent (rate, vacancy_control=False):
    controlled_rent = starting_rent
    market_rent = starting_rent
    real_rents = [] 
    tenancy_start = 0
    for m in all_months:
        market_rent = starting_rent*np.exp(m*cont_rate/12)
        if tenancy_end(m-tenancy_start)==True and vacancy_control == False:
            controlled_rent = market_rent
            tenancy_start = m
        elif ((m - tenancy_start) % 12) == 0:
            controlled_rent = min(controlled_rent + rate*controlled_rent,market_rent)
        real_rents.append(controlled_rent)
    return np.array(real_rents)

In [4]:
def run_simulation (rate,n=10000,vacancy_control=False):
    mean_rent = controlled_rent(rate,vacancy_control=vacancy_control)
    for i in range(n):
        mean_rent = mean_rent + controlled_rent(rate,vacancy_control=vacancy_control)
    return mean_rent/n

# mean_controlled = controlled_rent(new_rate)
# n = 10000
# for i in range(n):
#     mean_controlled = mean_controlled + controlled_rent(new_rate)
# mean_controlled = mean_controlled/n

controlled_exp = run_simulation(new_rate)

controlled_old_exp = run_simulation(old_rate)

controlled_vacancy_cpi = run_simulation(new_rate,vacancy_control=True)

controlled_vacancy_cpi_plus_two = run_simulation(old_rate,vacancy_control=True)

In [8]:
market_trace = go.Scatter(x=all_months,y=market_rents,name='Market')
trace1 = go.Scatter(x=all_months,y=controlled_old_exp,name='CPI +2%')
trace2 = go.Scatter(x=all_months,y=controlled_exp,name='CPI')
trace3 = go.Scatter(x=all_months,y=controlled_vacancy_cpi,name='CPI with vacancy control')
trace4 = go.Scatter(x=all_months,y=controlled_vacancy_cpi_plus_two,name='CPI + 2% with vacancy control')
data=[market_trace,trace1,trace2,trace3,trace4]
title = "MC Simulation of rents assuming annual market rates {:.2f}%, turnover {}%, cpi: {}%".format(100*market_rate,turnover_rate*1200,cpi*100)
layout=dict(title=title)
fig=dict(data=data,layout=layout)
iplot(fig)

print("CPI = {:.2f}%, Turnover = {:.2f}%, Market = {:.2f}%".format(100*cpi,turnover_rate*1200,market_rate*100))
print("Total rent under new policy: {}. Total rent under old policy: {}".format(np.sum(controlled_exp),np.sum(controlled_old_exp)))
print("Total difference between old and new policy: ${:.0f} ({:.2f}%)".format(np.sum(controlled_old_exp)-np.sum(controlled_exp),100*(np.sum(controlled_old_exp)-np.sum(controlled_exp))/np.sum(controlled_old_exp)))

CPI = 2.00%, Turnover = 20.00%, Market = 2.50%
Total rent under new policy: 266244.94495596056. Total rent under old policy: 269580.87806404254
Total difference between old and new policy: $3336 (1.24%)
